## To-do
- Now that we have import and metadata extraction working, we need to start preprocessing (mostly interpolating timepoints for z-slices if recorded on frame-by-frame basis by the scope) and some scheme for identification of a nuclear and a spot channel that is compatible with switching between the two channels (e.g. using mCherry to segment nuclei during cycles but not at the division).
- Makes sense to use dask for visualization (e.g. choosing a threshold).
- Write DoG/segmentation fuction so that it can take either 2D or 3D data - give the option to segment off of a projection, or off of raw 3D data.
    - Write in options for DoG and LoG segmentation algorithm with standard nuclear sizes vs box DoG/LoG vs watershed.
        - Actually, box filtering might not be very helpful if we're cutting off part of the nucleus is z - the BP filtering will project it into a distorted gaussian if we're not right in the middle of the nucleus, and then misplace the centroid and botch the diameter estimation from $\sigma$. For 3D segmentation, it might be better to use a single filter to find markers then perform a watershed.
- 3D DoG notes:
    - $\sigma_{x, y} = 8$ works perfectly to segment out nuclei during nc 13.
    - $\sigma_z$ is BP-filtered (1, 9) where 9 is the Z-sigma corresponding to the whole nucleus. This allow the BP to be very permissive in Z and filter out the nuclei in x and y.
- Proposed procedure for local peak finding:
    - Run box DoG as below with permissive BP in z and LoG approximation in (x, y), only varying $\sigma$ in the latter.
    - Peak-finding on standard image (e.g. $\sigma_{x, y} = 8$), then use coordinates as initial guess for next sigma values.
- Simple BP filter + peak finding does a good job finding markers. Give option then to watershed segment directly off of the image, off of distance-transformed otsu thresholded image, and off of edge-finding.
    - For data with the mid-nuclear plane on the boundary of our z-stack, might be useful to give the option to segment in 2D, then threshold each nuclear column locally to identify the nucleus.

In [1]:
import preprocessing.import_data as im

trim_series = True
lif_test_name = "test_data/2021-06-14/p2pdpwt"
lsm_test_name = "test_data/2023-04-07/p2pdp_zld-sites-ctrl_fwd_1"

(
    channels_full_dataset,
    original_global_metadata,
    original_frame_metadata,
    export_global_metadata,
    export_frame_metadata,
) = im.import_save_dataset(lsm_test_name, trim_series)

/home/ybadal/mambaforge/envs/transcription_pipeline/lib/python3.10/site-packages/pims/bioformats.py:410: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


In [2]:
import napari
import matplotlib.pyplot as plt
import numpy as np

In [3]:
nuclear_channel = channels_full_dataset[1]

In [14]:
import dask.array as da

image = nuclear_channel[40:50]
# nuclear_test = da.from_array(image, chunks=image.shape)

In [15]:
test_stack = image[4]

In [16]:
viewer = napari.view_image(test_stack, name="Nuclear Channel")
napari.run()

In [60]:
import numpy as np
from skimage import morphology as morph
from skimage.filters import difference_of_gaussians
from skimage.feature import peak_local_max
from scipy import ndimage as ndi


def _generate_cylinder(radius, height):
    """
    Constructs a cylindrical footprint for maximum dilation during peak finding by
    :func:`~skimage.feature.peak_local_max`.

    :param int radius: Radius of cylindrical footprint.
    :param int heigh: Height of cylindrical footprint.
    :return: Cylindrical footprint.
    :rtype: bool
    """
    disk = morph.disk(radius)
    cylinder = np.reshape(
        np.repeat(disk, height), (disk.shape[0], disk.shape[1], height)
    )
    return cylinder


def mark_nuclei(
    stack, low_sigma, high_sigma, footprint_radius, footprint_height, min_distance
):
    """
    Uses a difference of gaussians bandpass filter to enhance nuclei, then a local
    maximum to find markers for each nucleus. Being permissive with the filtering at
    this stage is recommended, since further filtering of the nuclear localization can
    be done post-segmentation using the size and morphology of the segmented objects.

    :param image: 2D (projected) or 3D image of a nuclear marker.
    :type image: Numpy array.
    :param float low_sigma: Sigma to use as the low-pass filter (mainly filters out
        noise).
    :param float high_sigma: Sigma to use as the high-pass filter (removes structured
        background and dims down areas where nuclei are close together that might
        start to coalesce under other morphological operations).
    :param int footprint_radius: Radius of cylindrical footprint used by
        :func:`~skimage.feature.peak_local_max` during maximum dilation.
    :param int footprint_height: Height of cylindrical footprint used by
        :func:`~skimage.feature.peak_local_max` during maximum dilation.
    :param int min_distance: minimum distance in pixels between nearby peaks.
    :return: Tuple(dog, marker_coordinates, markers) where dog is the
        bandpass-filtered image, marker_coordinates is an array of the nuclear
        locations in the image indexed as per the image (this can be used for
        visualization) and markers is a boolean array of the same shape as image, with
        the marker positions given by a True value.
    :rtype: Tuple of numpy arrays.
    """
    # Band-pass filter image using difference of gaussians - this seems to work
    # better than trying to do blob detection by varying sigma on an approximation of
    # a Laplacian of Gaussian filter.
    dog = difference_of_gaussians(stack, low_sigma=low_sigma, high_sigma=high_sigma)

    # Find local minima of the bandpass-filtered image to localize nuclei
    cylindrical_footprint = _generate_cylinder(footprint_radius, footprint_height)
    marker_coordinates = peak_local_max(
        dog,
        min_distance,
        exclude_border=True,
        footprint=cylindrical_footprint,
    )

    # Generate marker mask for segmentation downstream
    mask = np.zeros(dog.shape, dtype=bool)
    mask[tuple(marker_coordinates.T)] = True
    markers, _ = ndi.label(mask)

    return (dog, marker_coordinates, markers)

In [33]:
dog, marker_coords, marker_bool = mark_nuclei(
    test_stack, [1, 6, 6], [15, 10, 10], 2, 21, 4
)

In [19]:
viewer.add_points(marker_coords)

<Points layer 'marker_coords' at 0x7ff7d8270f10>

In [39]:
from skimage.morphology import ball
from skimage.filters import rank

normalized_image = test_stack / np.max(np.abs(test_stack))
denoised_stack = rank.median(normalized_image, ball(2))

In [41]:
viewer.add_image(denoised_stack)

<Image layer 'denoised_stack' at 0x7ff81b3f6140>

In [20]:
from skimage.segmentation import watershed

In [52]:
labels_raw = watershed(-denoised_stack, markers=marker_bool, mask=binarized_stack)

In [53]:
viewer.add_image(labels_raw, name="raw_watershed")

<Image layer 'raw_watershed' at 0x7ff819e83910>

In [48]:
from skimage.filters import threshold_otsu
from scipy import ndimage as ndi

threshold = threshold_otsu(denoised_stack)
binarized_stack = denoised_stack > threshold
distance = ndi.distance_transform_edt(binarized_stack)
labels_distance = watershed(-distance, markers=marker_bool, mask=binarized_stack)

In [49]:
viewer.add_image(labels_distance, name="distance_watershed")

<Image layer 'distance_watershed' at 0x7ff81c29beb0>

In [50]:
labels_dog = watershed(-dog, markers=marker_bool, mask=binarized_stack)

In [51]:
viewer.add_image(labels_dog, name="dog_watershed")

<Image layer 'dog_watershed' at 0x7ff819ef1e10>

In [57]:
from skimage.filters import sobel

elevation_map = sobel(denoised_stack)
labels_sobel = watershed(elevation_map, markers=marker_bool, mask=binarized_stack)

In [59]:
viewer.add_image(labels_sobel, name="sobel_watershed")

<Image layer 'sobel_watershed' at 0x7ff81c264880>

In [ ]:
from skimage.filters import rank
from skimage.filters import gaussian
from skimage.filters import threshold_otsu
from skimage.filters import threshold_li
from skimage.filters import sobel
from skimage.segmentation import watershed
from skimage.morphology import binary_closing
from scipy import ndimage as ndi


def segment_nuclei(
    stack,
    markers,
    denoising,
    thresholding,
    closing_footprint,
    watershed_method,
    **kwargs
):
    """
    Segments nuclei in a z-stack using watershed method.

    WRITE PARAMETER DOCUMENTATION HERE.
    """
    
    # Normalize image
    stack =/ np.max(np.abs(stack))

    # Denoising step
    if denoising == "gaussian":
        try:
            denoising_sigma = kwargs["denoising_sigma"]
        except KeyError:
            raise Exception("Gaussian denoising requires a denoising_sigma parameter.")
        denoised_stack = gaussian(stack, sigma=denoising_sigma)

    elif denoising == "median":
        try:
            median_footprint = kwargs["median_footprint"]
        except KeyError:
            raise Exception("Median denoising requires a median_footprint parameter.")
        denoised_stack = rank.median(stack, footprint=median_footprint)

    else:
        raise Exception("Unrecognized denoising parameter.")

    # Thresholding step
    if thresholding == "global_otsu":
        threshold = threshold_otsu(denoised_stack)

    elif thresholding == "local_otsu":
        try:
            otsu_footprint = kwargs["otsu_footprint"]
        except KeyError:
            raise Exception(
                "Local Otsu thresholding requires an otsu_footprint parameter."
            )
        threshold = rank.otsu(denoised_stack, otsu_footprint)

    elif thresholding == "li":
        threshold_guess = threshold_otsu(denoised_stack)
        threshold = threshold_li(denoised_stack, initial_guess=threshold_guess)

    else:
        raise Exception("Unrecognized thresholding parameter.")

    # Binarize stack by thresholding
    binarized_stack = denoised_stack >= threshold

    # Clean up binarized image with a closing operation
    binarized_stack = binary_closing(binarized_stack, closing_footprint)

    # Segmentation step
    if watershed_method == "raw":
        watershed_landscape = -denoised_stack

    elif watershed_method == "distance_transform":
        watershed_landscape = -(ndi.distance_transform_edt(binarized_stack))

    elif watershed_method == "sobel":
        watershed_landscape = sobel(denoised_stack)

    else:
        raise Exception("Unrecognized watershed_method parameter")

    labels = watershed(watershed_landscape, markers=markers, mask=binarized_stack)

    return labels